In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Embedding, Flatten
from tensorflow.keras.optimizers import Adam


In [2]:
# Load dataset
df = pd.read_csv('aggregated_data4.csv')

# Convert 'YearMonth' to datetime and extract features
df['YearMonth'] = pd.to_datetime(df['YearMonth'])
df['Year'] = df['YearMonth'].dt.year
df['Month'] = df['YearMonth'].dt.month
df.drop(['YearMonth'], axis=1, inplace=True)

# Separate target and features
y = df.pop('PESO DESEMBARCADO_KILOGRAMOS')
X = df

# Define the column transformer
numeric_features = ['SST', 'Year', 'Month']
numeric_transformer = StandardScaler()

categorical_features = ['NOMBRE PRINCIPAL', 'NOMBRE ESTADO', 'NOMBRE OFICINA']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Fit and transform the training data
X_processed = preprocessor.fit_transform(X)

# Split the processed data
X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=42)


In [3]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = x + inputs

    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return x + res

# Assuming X_train is now a numpy array after preprocessing
input_shape = X_train.shape[1:]  # This should now work since X_train is preprocessed
inputs = Input(shape=input_shape)
x = transformer_encoder(inputs, head_size=256, num_heads=4, ff_dim=4, dropout=0.25)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1)(x)

model = Model(inputs, x)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error')



IndexError: Exception encountered when calling layer 'softmax' (type Softmax).

tuple index out of range

Call arguments received by layer 'softmax' (type Softmax):
  • inputs=tf.Tensor(shape=(None, 4), dtype=float32)
  • mask=None

In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

# Evaluate the model
preds = model.predict(X_val)
mse = mean_squared_error(y_val, preds)
print(f'MSE: {mse}')